- We can access the raster data directly with `rioxarray`
- We need to apply the calibration factor from the metadata

In [ ]:
import rioxarray as riox
import xarray as xr
import dask.array as da
from pathlib import Path
import warnings
from rasterio.errors import NotGeoreferencedWarning
data_path = Path("/data/psp/SAN_FRANCISCO_ALOS1")

In [ ]:
# read metadata to extract the radiometric calibaration factor
def extract_calibration_factor(data_path):
    from math import sqrt, pow

    file_path = data_path / "ceos_leader.txt"
    with open(file_path, "r") as file:
        for line in file:
            if "Calibration Factor:" in line:
                parts = line.strip().split()
                # Assume the last element is the calibration value
                try:
                    value = float(parts[-1])
                    # dB to linear
                    value = sqrt(pow(10.0, (value - 32.0) / 10.0))
                    return value
                except ValueError("Invalid metadata."):
                    continue
    return None


calfac = extract_calibration_factor(data_path=data_path)

In [ ]:
# silence warnings when dataset is in the SAR geometry
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=NotGeoreferencedWarning)
    # open with chunks to use dask arrays
    S = (
        riox.open_rasterio(
            data_path / "VOL-ALPSRP202350750-P1.1__A",
            chunks=(-1, "auto", "auto"),
        )
        # TODO: benchmark with and without transposition
        .transpose("x", "y", "band").rename({"x": "x", "y": "y", "band": "elt"})
    )
# convert digital number to RCS
S *= calfac
# label axes
S.coords["elt"] = ["HH", "HV", "VH", "VV"]
# replace original metadata with descriptive tags
S.attrs = {"poltype": "S", "description": "Scattering matrix"}

In [ ]:
# inspect S
S

In [ ]:
# convert S to C3

# coordinates of the Hermitian covariance matrix
# we only store the upper part
C3_coords = (S.coords["y"], S.coords["x"], ["11", "12", "13", "22", "23", "33"])

# scattering vector
k_hh = S[:, :, 0]
k_vv = S[:, :, 3]
k_hv = 0.5 * (S[:, :, 1] + S[:, :, 2])

# compute the cross products
C3_elts = []
for i, e1 in enumerate([k_hh, k_hv, k_vv]):
    for j, e2 in enumerate([k_hh, k_hv, k_vv]):
        if j >= i:
            C3_elts.append(e1 * e2.conj())

# stack elements
C3_data = da.stack(C3_elts).T

# make a lazy DataArray
C3 = xr.DataArray(
    data=C3_data.rechunk(("auto", "auto", 6)),
    dims=("y", "x", "elt"),
    coords=C3_coords,
    name="C3",
    attrs={"poltype": "C3", "description": "Covariance matrix"},
)

In [ ]:
# inspect C3
C3

In [ ]:
# multilooking in azimuth
C3_mlt = C3.coarsen(y=8).mean()

In [ ]:
# plot the data
abs(C3_mlt.loc[..., "33"]).plot.imshow(interpolation="none", vmax=1)